In [23]:
# predict the model  video cap

from ultralytics import YOLO
import cv2
import os

# Load the trained model
model = YOLO(r"C:\Users\pavan\Downloads\yolo_IND\runs\train\defect_detector6\weights\best.pt")

# Ask user whether to save video
save_video = input("Do you want to save the detected video? (y/n): ").strip().lower() == 'y'

# Use webcam (0 = default camera)
cap = cv2.VideoCapture(0)

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS)) or 20  # fallback if FPS not detected

# Set up video writer if saving is enabled
if save_video:
    output_path = "detected_output.avi"
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))
    print(f"📹 Saving video to: {os.path.abspath(output_path)}")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLO detection
    results = model(frame)
    annotated = results[0].plot()  # Draw bounding boxes

    # Write frame if saving is enabled
    if save_video:
        out.write(annotated)

    cv2.imshow("Industrial Defect Detection", annotated)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
if save_video:
    out.release()
cv2.destroyAllWindows()

print("✅ Detection finished.")
if save_video:
    print(f"🎬 Video saved at: {os.path.abspath(output_path)}")


Do you want to save the detected video? (y/n):  y


📹 Saving video to: C:\Users\pavan\Downloads\yolo_IND\detected_output.avi

0: 96x128 1 patches, 37.9ms
Speed: 1.1ms preprocess, 37.9ms inference, 2.3ms postprocess per image at shape (1, 3, 96, 128)

0: 96x128 1 patches, 61.4ms
Speed: 13.6ms preprocess, 61.4ms inference, 1.3ms postprocess per image at shape (1, 3, 96, 128)

0: 96x128 1 patches, 36.1ms
Speed: 8.9ms preprocess, 36.1ms inference, 2.3ms postprocess per image at shape (1, 3, 96, 128)

0: 96x128 1 patches, 39.9ms
Speed: 3.2ms preprocess, 39.9ms inference, 2.3ms postprocess per image at shape (1, 3, 96, 128)

0: 96x128 1 patches, 33.6ms
Speed: 1.5ms preprocess, 33.6ms inference, 3.0ms postprocess per image at shape (1, 3, 96, 128)

0: 96x128 1 patches, 42.8ms
Speed: 1.7ms preprocess, 42.8ms inference, 1.3ms postprocess per image at shape (1, 3, 96, 128)

0: 96x128 1 patches, 41.0ms
Speed: 1.1ms preprocess, 41.0ms inference, 4.8ms postprocess per image at shape (1, 3, 96, 128)

0: 96x128 1 patches, 34.0ms
Speed: 1.8ms preproces

In [1]:
# testing image 

from ultralytics import YOLO
from PIL import Image
import os

# Load trained model
model = YOLO(r"C:\Users\pavan\Downloads\yolo_IND\runs\train\defect_detector6\weights\best.pt")

# Image path (single image file, not folder)
img_path = r"C:\Users\pavan\Downloads\yolo_IND\test\images\scratches_7.jpg"  # <-- change filename here

# Run detection
results = model(img_path)

# Access first result
result = results[0]

# Display image with bounding boxes
result.show()

# Save annotated image
output_dir = "output"
os.makedirs(output_dir, exist_ok=True)
# result.save(filename=os.p ath.join(output_dir, "detected_sample.jpg"))

# Print detected classes and confidence
print("\n📋 Detected Objects:")
for box in result.boxes:
    cls_id = int(box.cls[0])  # class ID
    conf = float(box.conf[0])  # confidence score
    class_name = model.names[cls_id]  # get class label
    print(f"🔹 Class: {class_name} | Confidence: {conf*100:.2f}%")

print("\n✅ Detection complete! Saved annotated image to output/detected_sample.jpg")




image 1/1 C:\Users\pavan\Downloads\yolo_IND\test\images\scratches_7.jpg: 128x128 1 inclusion, 1 scratches, 120.5ms
Speed: 8.8ms preprocess, 120.5ms inference, 23.3ms postprocess per image at shape (1, 3, 128, 128)

📋 Detected Objects:
🔹 Class: scratches | Confidence: 91.74%
🔹 Class: inclusion | Confidence: 30.01%

✅ Detection complete! Saved annotated image to output/detected_sample.jpg


In [13]:
#convertion to xmt to txt

import os
from PIL import Image

# Define your dataset root
dataset_path = r"C:\Users\pavan\Downloads\yolo_IND\data\validation\images"   # change this to your dataset folder
output_images_dir = "new_data_val/train/images"
output_labels_dir = "new_data_val/train/labels"

# Define class names and IDs
class_names = ['crazing', 'inclusion', 'patches', 'pitted_surface', 'rolled_in_scale', 'scratches']
class_to_id = {cls: i for i, cls in enumerate(class_names)}

# Create output folders
os.makedirs(output_images_dir, exist_ok=True)
os.makedirs(output_labels_dir, exist_ok=True)

# Loop through each class folder
for cls_name in class_names:
    cls_folder = os.path.join(dataset_path, cls_name)
    if not os.path.exists(cls_folder):
        print(f"⚠️ Skipping missing folder: {cls_folder}")
        continue

    cls_id = class_to_id[cls_name]

    for file in os.listdir(cls_folder):
        if file.lower().endswith(('.jpg', '.jpeg', '.png')):
            img_path = os.path.join(cls_folder, file)

            # Copy image to output_images_dir
            img = Image.open(img_path)
            w, h = img.size

            # Create corresponding .txt label file
            label_path = os.path.join(output_labels_dir, file.rsplit('.', 1)[0] + '.txt')

            # Use full image box (0.5,0.5,1,1) if you don't have bounding boxes
            # Replace this if you have real coordinates
            with open(label_path, 'w') as f:
                f.write(f"{cls_id} 0.5 0.5 1 1\n")

            # Optionally copy image to output_images_dir
            img.save(os.path.join(output_images_dir, file))

print("✅ All label .txt files generated successfully!")
print(f"Labels saved to: {output_labels_dir}")


⚠️ Skipping missing folder: C:\Users\pavan\Downloads\yolo_IND\data\validation\images\rolled_in_scale
✅ All label .txt files generated successfully!
Labels saved to: new_data_val/train/labels
